In [1]:
from dotenv import load_dotenv
import time
load_dotenv()
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import OpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
gpt_turbo = ChatOpenAI(model_name="gpt-3.5-turbo")

In [3]:
path = "data/Vigilance Table.csv"
loader = CSVLoader(file_path=path)
data = loader.load()


In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(data)

In [5]:
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [6]:
retriever = vectorstore.as_retriever()

In [7]:
with open("contextualize_system_prompt.txt", "r") as files:
    contextualize_system_prompt = files.read()

with open("instruct.txt", "r") as files:
    instruct = files.read()

In [8]:
contextualize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)
contextualize_prompt.pretty_print()
contextualize_chain = contextualize_prompt | gpt_turbo | StrOutputParser()

================================ System Message ================================

Given a chat history and the latest user question which might reference context in the chat history, formulate a standalone question just reformulate it if needed and otherwise return it as is.


============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{question}


In [10]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", instruct),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)
prompt.pretty_print()

def contextualized_question(input: dict):
    if input.get("chat_history"):
        return contextualize_chain
    else:
        return input["question"]


rag_chain = (
    RunnablePassthrough.assign(
        context=contextualized_question | retriever
    )
    | prompt
    | gpt_turbo
)

================================ System Message ================================

Tu es un chatbot spécialiser sur les conseils liés aux intempéries qui ne répond UNIQUEMENT qu'en FRANÇAIS, tu peux t'aider du context suivant pour répondre : {context}


============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{question}


In [11]:
chat_history = []
question = "Comment se protéger de la canicule en vigilance orange ?"
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
print(ai_msg.content)

Pour se protéger de la canicule en vigilance orange, voici quelques conseils :

1. Limitez vos sorties, surtout le soir, la nuit et en début de matinée.
2. Habillez-vous chaudement en superposant plusieurs couches de vêtements. Portez une couche extérieure imperméable au vent et à l'eau, couvrez-vous la tête et les mains, et évitez de garder des vêtements humides.
3. Après être rentré à l'intérieur, assurez-vous de vous reposer suffisamment, prenez une douche ou un bain chaud, alimentez-vous convenablement et buvez des boissons chaudes (évitez l'alcool).
4. Faites attention aux moyens de chauffage utilisés. Les chauffages d'appoint ne doivent pas fonctionner en continu, et ne jamais utilisez des cuisinières, braséros, etc. pour vous chauffer. Ne bloquez pas les entrées d'air de votre logement.
5. Aérez votre logement quelques minutes même en hiver.
6. Évitez les efforts brusques.

N'hésitez pas à consulter les sites officiels pour plus d'informations :
- https://sante.gouv.fr/
- https:

In [12]:
chat_history = []
question = "Que dois-je faire dans une zone inondable"
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
print(ai_msg.content)

Si vous vous trouvez dans une zone inondable, voici quelques conseils à suivre :

- Restez chez vous et tenez-vous informé auprès des autorités.
- Éloignez-vous des cours d'eau, des points bas et des ponts, et rejoignez le point le plus haut possible.
- Évitez d'utiliser votre voiture.
- Ne cherchez pas à récupérer vos enfants à l'école.
- Réfugiez-vous en étage, et en dernier recours sur le toit, sans descendre dans les sous-sols.
- Évacuez uniquement sur ordre des autorités en emportant votre kit d'urgence.

Ces mesures de précaution peuvent vous aider à rester en sécurité en cas d'inondation.


In [13]:
chat_history = [HumanMessage(content=question), ai_msg]
question = ("comment me tenir informé de l'évolution")
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
print(ai_msg.content)

Pour vous tenir informé de l'évolution de la situation en cas d'intempéries, voici quelques recommandations :

- Suivez les informations diffusées par les autorités compétentes, telles que la préfecture, la mairie ou les services météorologiques.
- Consultez régulièrement les bulletins météorologiques et les alertes de vigilance.
- Restez à l'écoute des médias locaux (radio, télévision) pour des mises à jour en temps réel.
- Utilisez les applications mobiles dédiées aux alertes météorologiques pour recevoir des notifications.
- Restez en contact avec vos proches pour vous tenir mutuellement informés de la situation.

En restant attentif aux informations officielles et en suivant les consignes de sécurité, vous pourrez mieux gérer les intempéries et protéger votre sécurité.
